In [1]:
import os
import sys
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

sys.path.append("/home/bongo/porter_notebook/research/qwen3")

# config 및 file_managing에서 필요한 변수/함수 임포트
from config import AGD20K_PATH, model_name
from file_managing import make_input_image

# ------------------------------------------------------
# 1. 모델 로딩
# ------------------------------------------------------
# SDPA를 끄면 메모리를 좀 더 쓰지만 Attention Map은 확실히 나옵니다.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTORCH_ENABLE_SDPA"] = "0"  # 1 -> 0 변경
os.environ["PYTORCH_ENABLE_FLASH_SDP"] = "0"
os.environ["PYTORCH_ENABLE_MEM_EFFICIENT_SDP"] = "0"

print(f"🤖 Loading {model_name} for selection...")
model = Qwen3VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    device_map="cuda", 
)

processor = AutoProcessor.from_pretrained(model_name)
device = model.device

/home/bongo/anaconda3/envs/qwen3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [2]:
import os
import pandas as pd

In [3]:
# 이전 단계에서 저장한 최적의 Exo 이미지 DB 로드
EXO_DB_PATH = "32B_selected_best_exo_images.pkl" 
if not os.path.exists(EXO_DB_PATH):
    raise FileNotFoundError(f"Exo DB를 먼저 생성해야 합니다: {EXO_DB_PATH}")
df_exo_db = pd.read_pickle(EXO_DB_PATH)
df_exo_db

,action,object,best_exo_path,filename,score,output_text
0,beat,drum,/root/AGD20K/Seen/trainset/exocentric/beat/dru...,beat_drum_004242.jpg,416.964844,playing \ntapping \nbeating \nstriking \nd...
1,boxing,punching_bag,/root/AGD20K/Seen/trainset/exocentric/boxing/p...,boxing_punching_bag_002857.jpg,394.583984,punching \nhitting \nboxing \nkicking \ntr...
2,brush_with,toothbrush,/root/AGD20K/Seen/trainset/exocentric/brush_wi...,brush_with_toothbrush_004655.jpg,235.555339,brushing \nholding \nusing \ncleaning \nsm...
3,carry,skateboard,/root/AGD20K/Seen/trainset/exocentric/carry/sk...,carry_skateboard_009347.jpg,431.921875,holding \nstanding \ncarrying \nposing \ng...
4,carry,skis,/root/AGD20K/Seen/trainset/exocentric/carry/sk...,carry_skis_000436.jpg,343.231445,"holding, carrying, gripping, lifting, supporti..."
...,...,...,...,...,...,...
99,wash,fork,/root/AGD20K/Seen/trainset/exocentric/wash/for...,wash_fork_004874.jpg,289.328125,rinsing \nwashing \nholding \ncleaning \ns...
100,wash,knife,/root/AGD20K/Seen/trainset/exocentric/wash/kni...,wash_knife_009591.jpg,324.894531,rinsing \nwashing \ncleaning \nholding \nr...
101,wash,orange,/root/AGD20K/Seen/trainset/exocentric/wash/ora...,wash_orange_000228.jpg,174.287109,washing \nrinsing \nholding \nscrubbing \n...
102,wash,toothbrush,/root/AGD20K/Seen/trainset/exocentric/wash/too...,wash_toothbrush_006501.jpg,360.039062,rinsing \nwashing \ncleaning \nbrushing \n...


In [2]:
image_path, target_action, object_name = "/home/DATA/AGD20K/Seen/trainset/exocentric/carry/surfboard/carry_surfboard_001939.jpg", "carry", "surfboard"

In [3]:

# -----------------------------------------------------------
# 1. 전처리: 핵심 동사 추출
# -----------------------------------------------------------
# 사전 없이 단순 split만 사용
core_action = target_action.split('_')[0].lower() 

# -----------------------------------------------------------
# 2. 질문: 동사 나열 유도
# -----------------------------------------------------------
query = f"What actions is the person doing with the {object_name}? list all the possible verbs. Only list the verbs."


image_base64 = make_input_image(str(image_path))


messages = [
    {"role": "user", "content": [
        {"type": "image", "image": f"data:image/jpeg;base64,{image_base64}"},
        {"type": "text", "text": query}
    ]}
]
inputs = processor.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_dict=True, return_tensors="pt").to(model.device)

# Vision Token 인덱스 찾기
input_ids_list = inputs.input_ids[0].tolist()
vis_start_id = processor.tokenizer.convert_tokens_to_ids("<|vision_start|>")
vis_end_id = processor.tokenizer.convert_tokens_to_ids("<|vision_end|>")

vis_start_idx = input_ids_list.index(vis_start_id)
vis_end_idx = input_ids_list.index(vis_end_id)

with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=30, output_attentions=True, return_dict_in_generate=True)
    
output_ids = generated_ids.sequences[0][inputs.input_ids.shape[1]:]
full_text = processor.decode(output_ids, skip_special_tokens=True).lower()
full_text

'carry'

In [4]:
output_ids

tensor([ 73765, 151645], device='cuda:0')

In [14]:


# -----------------------------------------------------------
# 5. Targeted Scoring (Safety Net 추가)
# -----------------------------------------------------------
total_vis_score = 0.0     # 매칭된 토큰들의 합
matched_count = 0         # 매칭된 토큰 수

all_tokens_energy_sum = 0.0 # (Safety Net용) 전체 토큰 에너지 합
valid_token_count = 0       # (Safety Net용) 전체 유효 토큰 수

for i, token_id in enumerate(output_ids):
    token_str = processor.decode([token_id], skip_special_tokens=True).lower().strip()
    
    if not token_str: continue # 공백 토큰 무시

    # --- [에너지 계산] ---
    # 매칭 여부와 상관없이 일단 현재 토큰의 에너지를 계산해둡니다.
    token_energy = 0.0
    # ... 인덱스 찾는 부분 직후 ...
    print(f"🔍 [Debug Indices] Start: {vis_start_idx}, End: {vis_end_idx}")
    print(f"🔍 [Debug Length] Image Tokens Count: {vis_end_idx - vis_start_idx - 1}")

    for layer_attn in generated_ids.attentions[i]:
        # Vision Token 영역만 슬라이싱하여 합산
        vision_attn = layer_attn[0, :, 0, vis_start_idx+1 : vis_end_idx]
        token_energy += vision_attn.float().sum().item()
        # print(f"token_energy : {token_energy}")
    
    # Safety Net을 위해 전체 누적
    all_tokens_energy_sum += token_energy
    valid_token_count += 1

    # --- [토큰 매칭 확인] ---
    # 1. Core Action이 Token을 포함 (예: carry >= car) -> BPE 파편화 대응
    # 2. Token이 Core Action을 포함 (예: carrying >= carry) -> 변형 대응
    if (core_action in token_str) or (token_str in core_action and len(token_str) > 1): 
        # print("good")
        # len > 1 조건: 'c', 'a' 같은 너무 짧은 파편이 엄한 단어에 매칭되는 것 방지
        total_vis_score += token_energy
        matched_count += 1
        # print(a)
    print(f"Token {i}: {token_str}  / :{total_vis_score}")
# -----------------------------------------------------------
# [결과 반환 로직]
# -----------------------------------------------------------
if matched_count > 0:
    # 1. 정확히 매칭된 토큰이 있으면 그 점수 사용 (Best)
    final_score = total_vis_score / matched_count
    print("FIN :",  final_score, full_text )
else:
    # 2. [Safety Net] 매칭된 토큰은 없지만, full_text에는 정답이 있었음!
    # 토크나이징 문제로 판단하고, 전체 문장의 평균 에너지를 반환 (Fallback)
    if valid_token_count > 0:
        final_score = all_tokens_energy_sum / valid_token_count
        print("FIN2 :",  final_score, full_text )
    else:
        print("FIN3 :",  0.0, full_text )# 토큰이 없으면 0점

print( final_score, full_text)


🔍 [Debug Indices] Start: 3, End: 965
🔍 [Debug Length] Image Tokens Count: 961
Token 0: carry  / :0.0
FIN : 0.0 carry
0.0 carry


In [13]:
layer_attn[0, :, 0, : ]

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.bfloat16)

In [7]:
print(f"🔍 Current Attention Implementation: {model.config}")

🔍 Current Attention Implementation: Qwen3VLConfig {
  "architectures": [
    "Qwen3VLForConditionalGeneration"
  ],
  "dtype": "bfloat16",
  "image_token_id": 151655,
  "model_type": "qwen3_vl",
  "text_config": {
    "attention_bias": false,
    "attention_dropout": 0.0,
    "bos_token_id": 151643,
    "dtype": "bfloat16",
    "eos_token_id": 151645,
    "head_dim": 128,
    "hidden_act": "silu",
    "hidden_size": 2048,
    "initializer_range": 0.02,
    "intermediate_size": 6144,
    "max_position_embeddings": 262144,
    "model_type": "qwen3_vl_text",
    "num_attention_heads": 16,
    "num_hidden_layers": 28,
    "num_key_value_heads": 8,
    "rms_norm_eps": 1e-06,
    "rope_scaling": {
      "mrope_interleaved": true,
      "mrope_section": [
        24,
        20,
        20
      ],
      "rope_type": "default"
    },
    "rope_theta": 5000000,
    "tie_word_embeddings": true,
    "use_cache": true,
    "vocab_size": 151936
  },
  "tie_word_embeddings": true,
  "transformers_v

In [6]:
import numpy as np

In [12]:
layer_attn[0, :, 0, :].sum()

tensor(16., device='cuda:0', dtype=torch.bfloat16)

In [10]:
np.shape(layer_attn[0, :, 0, :])

torch.Size([16, 993])

In [9]:
token_energy

0.0

In [1]:
import pandas as pd

In [4]:
df = pd.read_pickle("selected_best_exo_images.pkl")
df

,action,object,best_exo_path,filename,score,output_text
0,beat,drum,/home/DATA/AGD20K/Seen/trainset/exocentric/bea...,beat_drum_005505.jpg,55.248047,"play, hold, strike, beat"
1,carry,skateboard,/home/DATA/AGD20K/Seen/trainset/exocentric/car...,carry_skateboard_014049.jpg,63.373047,"holding, carrying, walking, standing, holding ..."
2,carry,skis,/home/DATA/AGD20K/Seen/trainset/exocentric/car...,carry_skis_002683.jpg,51.976562,"holding, carrying, standing, walking, skiing, ..."
3,carry,snowboard,/home/DATA/AGD20K/Seen/trainset/exocentric/car...,carry_snowboard_006749.jpg,65.386719,"hold, carry, adjust, move, stand, push, lift, ..."
4,carry,surfboard,/home/DATA/AGD20K/Seen/trainset/exocentric/car...,carry_surfboard_001850.jpg,23.713379,"carry, hold, stand, support, carry, hold, support"
...,...,...,...,...,...,...
88,wash,knife,/home/DATA/AGD20K/Seen/trainset/exocentric/was...,wash_knife_013931.jpg,28.540039,"wash, hold, rinse, clean, wipe, spray, rinse, ..."
89,wash,orange,/home/DATA/AGD20K/Seen/trainset/exocentric/was...,wash_orange_003857.jpg,26.551432,"wash, hold, squeeze, peel, scrub, rinse, clean..."
90,wash,toothbrush,/home/DATA/AGD20K/Seen/trainset/exocentric/was...,wash_toothbrush_011680.jpg,56.177734,"brush, clean, rinse, wash"
91,wash,wine_glass,/home/DATA/AGD20K/Seen/trainset/exocentric/was...,wash_wine_glass_013676.jpg,64.607422,"fill, wash, clean, rinse, pour"


In [5]:
import os
from collections import defaultdict
from tqdm import tqdm

# ---------------------------------------------------------
# [설정] 데이터셋 경로
# ---------------------------------------------------------
DATASET_ROOT = '/home/DATA/AGD20K/Seen/trainset/exocentric'
OUTPUT_FILE = 'exo_unique.txt'

def get_image_id(filename):
    """
    파일명에서 고유 ID 추출 함수
    예: 'brush_with_toothbrush_000006.jpg' -> '000006.jpg'
    (맨 뒤의 _ 기준 뒷부분을 ID로 사용)
    """
    return filename.split('_')[-1]

def log_and_print(f, text):
    """화면 출력과 파일 저장을 동시에 수행"""
    print(text)
    f.write(text + "\n")

def analyze_unique_images_fast(root_path, output_path):
    # 1. 데이터 구조
    # global_id_map: 이미지ID -> 이 이미지가 발견된 (action, object) 쌍들의 집합(Set)
    global_id_map = defaultdict(set)
    
    # pair_files: (action, object) -> 해당 쌍이 가진 이미지ID 리스트
    pair_files = defaultdict(list)
    
    print(f"📂 Scanning directory (Fast Mode): {root_path}")

    # 2. 디렉토리 순회 (파일명만 수집)
    if not os.path.exists(root_path):
        print(f"Error: Path not found - {root_path}")
        return

    # Action 폴더 순회
    actions = sorted([d for d in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, d))])
    
    for action in tqdm(actions, desc="Scanning Actions"):
        action_path = os.path.join(root_path, action)
        
        # Object 폴더 순회
        objects = sorted([d for d in os.listdir(action_path) if os.path.isdir(os.path.join(action_path, d))])
        
        for obj in objects:
            obj_path = os.path.join(action_path, obj)
            
            # 파일 리스트 가져오기
            files = [f for f in os.listdir(obj_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            for filename in files:
                # 파일명에서 ID 추출 (MD5 대신 사용)
                img_id = get_image_id(filename)
                
                # 메타데이터 기록
                pair_files[(action, obj)].append(img_id)
                global_id_map[img_id].add((action, obj))

    print("📊 Calculating uniqueness...")

    # 3. 결과 집계 및 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        # 헤더 작성
        header = f"{'Action':<25} | {'Object':<20} | {'Total':<8} | {'Unique':<8} | {'Overlap':<8} | {'Unique%':<8}"
        sep_line = "-" * len(header)
        
        log_and_print(f, f"Results saved to: {output_path}")
        log_and_print(f, sep_line)
        log_and_print(f, header)
        log_and_print(f, sep_line)
        
        total_files_count = 0
        total_unique_count = 0
        
        # Action, Object 순으로 정렬하여 출력
        sorted_pairs = sorted(pair_files.keys())
        
        for action, obj in sorted_pairs:
            img_ids = pair_files[(action, obj)]
            total = len(img_ids)
            
            unique = 0
            for img_id in img_ids:
                # 이 ID를 가진 (action, object) 쌍이 1개뿐이면 Unique
                if len(global_id_map[img_id]) == 1:
                    unique += 1
            
            overlap = total - unique
            unique_rate = (unique / total * 100) if total > 0 else 0
            
            total_files_count += total
            total_unique_count += unique
            
            line = f"{action:<25} | {obj:<20} | {total:<8} | {unique:<8} | {overlap:<8} | {unique_rate:6.1f}%"
            log_and_print(f, line)
            
        log_and_print(f, sep_line)
        log_and_print(f, f"Grand Total Files: {total_files_count}")
        # 주의: 여기서 Total Unique Files는 '겹치지 않는 파일의 총 개수'입니다.
        log_and_print(f, f"Total Unique Files (Non-overlapping): {total_unique_count}")
        log_and_print(f, sep_line)

if __name__ == "__main__":
    analyze_unique_images_fast(DATASET_ROOT, OUTPUT_FILE)

📂 Scanning directory (Fast Mode): /home/DATA/AGD20K/Seen/trainset/exocentric


Scanning Actions:   0%|          | 0/36 [00:00<?, ?it/s]

Scanning Actions: 100%|██████████| 36/36 [00:00<00:00, 569.65it/s]

📊 Calculating uniqueness...
Results saved to: exo_unique.txt
--------------------------------------------------------------------------------------------
Action                    | Object               | Total    | Unique   | Overlap  | Unique% 
--------------------------------------------------------------------------------------------
beat                      | drum                 | 116      | 116      | 0        |  100.0%
boxing                    | punching_bag         | 373      | 373      | 0        |  100.0%
brush_with                | toothbrush           | 392      | 211      | 181      |   53.8%
carry                     | skateboard           | 76       | 11       | 65       |   14.5%
carry                     | skis                 | 33       | 0        | 33       |    0.0%
carry                     | snowboard            | 30       | 6        | 24       |   20.0%
carry                     | surfboard            | 220      | 94       | 126      |   42.7%
catch           

In [9]:
df[df["action"]=="brush_with"]

,action,object,filename,full_path,gt_path,Exp1_Context_LastInput_KLD,Exp1_Context_LastInput_SIM,Exp1_Context_LastInput_NSS,Exp2_Context_AvgOutput_KLD,Exp2_Context_AvgOutput_SIM,...,Exp9_EgoOnly_NSS,Exp10_EgoOnly_KLD,Exp10_EgoOnly_SIM,Exp10_EgoOnly_NSS,Exp11_EgoOnly_KLD,Exp11_EgoOnly_SIM,Exp11_EgoOnly_NSS,Exp12_EgoOnly_KLD,Exp12_EgoOnly_SIM,Exp12_EgoOnly_NSS
35,brush_with,toothbrush,toothbrush_000097.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,brush_with,toothbrush,toothbrush_000176.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,brush_with,toothbrush,toothbrush_000427.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,brush_with,toothbrush,toothbrush_000575.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,brush_with,toothbrush,toothbrush_001596.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,brush_with,toothbrush,toothbrush_001764.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,brush_with,toothbrush,toothbrush_001991.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,brush_with,toothbrush,toothbrush_002109.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,brush_with,toothbrush,toothbrush_002342.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,brush_with,toothbrush,toothbrush_002589.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/brus...,/home/DATA/AGD20K/Seen/testset/GT/brush_with/t...,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_coocur['jump_skis'][0]

'/home/DATA/AGD20K/Seen/trainset/exocentric/jump/skis/jump_skis_002294.jpg'

In [ ]:
overlap_first_exo_candidates.pkl

In [16]:
df_coocur = pd.read_pickle("/home/bongo/porter_notebook/research/qwen3/ECCV_codes/ablations/Seen/overlap_first_exo_candidates.pkl")
df_coocur

{'jump_skis': ['/home/DATA/AGD20K/Seen/trainset/exocentric/jump/skis/jump_skis_006213.jpg'],
 'jump_skateboard': ['/home/DATA/AGD20K/Seen/trainset/exocentric/jump/skateboard/jump_skateboard_013740.jpg'],
 'jump_surfboard': ['/home/DATA/AGD20K/Seen/trainset/exocentric/jump/surfboard/jump_surfboard_004998.jpg'],
 'jump_snowboard': ['/home/DATA/AGD20K/Seen/trainset/exocentric/jump/snowboard/jump_snowboard_007003.jpg'],
 'peel_carrot': ['/home/DATA/AGD20K/Seen/trainset/exocentric/peel/carrot/peel_carrot_010737.jpg'],
 'peel_orange': ['/home/DATA/AGD20K/Seen/trainset/exocentric/peel/orange/peel_orange_002110.jpg'],
 'peel_banana': ['/home/DATA/AGD20K/Seen/trainset/exocentric/peel/banana/peel_banana_009284.jpg'],
 'peel_apple': ['/home/DATA/AGD20K/Seen/trainset/exocentric/peel/apple/peel_apple_006922.jpg'],
 'wash_toothbrush': ['/home/DATA/AGD20K/Seen/trainset/exocentric/wash/toothbrush/wash_toothbrush_011680.jpg'],
 'wash_cup': ['/home/DATA/AGD20K/Seen/trainset/exocentric/wash/cup/wash_cup_

In [15]:
df_nocoocur = pd.read_pickle("/home/bongo/porter_notebook/research/qwen3/ECCV_codes/ablations/Seen/unique_first_exo_candidates.pkl")
df_nocoocur

{'jump_skis': ['/home/DATA/AGD20K/Seen/trainset/exocentric/jump/skis/jump_skis_002294.jpg'],
 'jump_skateboard': ['/home/DATA/AGD20K/Seen/trainset/exocentric/jump/skateboard/jump_skateboard_008357.jpg'],
 'jump_surfboard': ['/home/DATA/AGD20K/Seen/trainset/exocentric/jump/surfboard/jump_surfboard_002567.jpg'],
 'jump_snowboard': ['/home/DATA/AGD20K/Seen/trainset/exocentric/jump/snowboard/jump_snowboard_012480.jpg'],
 'peel_carrot': ['/home/DATA/AGD20K/Seen/trainset/exocentric/peel/carrot/peel_carrot_010090.jpg'],
 'peel_orange': ['/home/DATA/AGD20K/Seen/trainset/exocentric/peel/orange/peel_orange_000504.jpg'],
 'peel_banana': ['/home/DATA/AGD20K/Seen/trainset/exocentric/peel/banana/peel_banana_001926.jpg'],
 'peel_apple': ['/home/DATA/AGD20K/Seen/trainset/exocentric/peel/apple/peel_apple_004215.jpg'],
 'wash_toothbrush': ['/home/DATA/AGD20K/Seen/trainset/exocentric/wash/toothbrush/wash_toothbrush_006501.jpg'],
 'wash_cup': ['/home/DATA/AGD20K/Seen/trainset/exocentric/wash/cup/wash_cup_

In [6]:
df = pd.read_pickle("/home/bongo/porter_notebook/research/qwen3/ECCV_codes/ablations/Seen/2B_attention_result_seen_all.pkl")
df

,action,object,filename,full_path,gt_path,Exp1_Context_LastInput_KLD,Exp1_Context_LastInput_SIM,Exp1_Context_LastInput_NSS,Exp2_Context_AvgOutput_KLD,Exp2_Context_AvgOutput_SIM,...,Exp9_EgoOnly_NSS,Exp10_EgoOnly_KLD,Exp10_EgoOnly_SIM,Exp10_EgoOnly_NSS,Exp11_EgoOnly_KLD,Exp11_EgoOnly_SIM,Exp11_EgoOnly_NSS,Exp12_EgoOnly_KLD,Exp12_EgoOnly_SIM,Exp12_EgoOnly_NSS
0,beat,drum,drum_000346.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/beat...,/home/DATA/AGD20K/Seen/testset/GT/beat/drum/dr...,None,None,None,None,None,...,0.639215,0.257249,0.731574,0.979651,0.257249,0.731574,0.979651,0.257249,0.731574,0.979651
1,beat,drum,drum_000397.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/beat...,/home/DATA/AGD20K/Seen/testset/GT/beat/drum/dr...,None,None,None,None,None,...,1.744280,0.557498,0.595862,2.064037,0.557498,0.595862,2.064037,0.557498,0.595862,2.064037
2,beat,drum,drum_000415.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/beat...,/home/DATA/AGD20K/Seen/testset/GT/beat/drum/dr...,None,None,None,None,None,...,0.534109,0.414922,0.636499,0.774356,0.355733,0.664948,0.821385,0.414922,0.636499,0.774356
3,beat,drum,drum_000899.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/beat...,/home/DATA/AGD20K/Seen/testset/GT/beat/drum/dr...,None,None,None,None,None,...,0.288482,0.190259,0.779504,0.441353,0.171027,0.791240,0.463379,0.190259,0.779504,0.441353
4,beat,drum,drum_001259.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/beat...,/home/DATA/AGD20K/Seen/testset/GT/beat/drum/dr...,None,None,None,None,None,...,1.258344,0.655866,0.544514,1.689589,0.655866,0.544514,1.689589,0.655866,0.544514,1.689589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,write,pen,pen_003441.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/writ...,/home/DATA/AGD20K/Seen/testset/GT/write/pen/pe...,None,None,None,None,None,...,0.441123,1.670467,0.290382,1.240493,1.670467,0.290382,1.240493,1.865011,0.235834,0.864358
1706,write,pen,pen_003526.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/writ...,/home/DATA/AGD20K/Seen/testset/GT/write/pen/pe...,None,None,None,None,None,...,2.826656,0.961174,0.421612,2.990695,1.135167,0.363498,2.696913,0.748262,0.495059,3.443997
1707,write,pen,pen_003590.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/writ...,/home/DATA/AGD20K/Seen/testset/GT/write/pen/pe...,None,None,None,None,None,...,1.009173,1.619796,0.242319,0.989857,1.619796,0.242319,0.989857,1.293049,0.313086,1.532452
1708,write,pen,pen_003638.jpg,/home/DATA/AGD20K/Seen/testset/egocentric/writ...,/home/DATA/AGD20K/Seen/testset/GT/write/pen/pe...,None,None,None,None,None,...,2.132784,0.847918,0.458087,2.643255,0.847918,0.458087,2.643255,0.753843,0.489499,2.668594
